# NYC Property Risk Assessment — EDA & Feature Engineering

This notebook explores the geospatial datasets used in the multimodal RAG pipeline and engineers location-based risk features that improve retrieval relevance and LLM reasoning quality.

**Datasets:**
- `New_York_City_s_Flood_Vulnerability_Index.csv` — per-tract FVI scores (storm surge, tidal, FSHRI)
- `nyct2020_25a/nyct2020.shp` — NYC 2020 census tract boundaries
- `nybb_25a/nybb.shp` — NYC borough boundaries
- NYPD complaint data — fetched live via NYC Open Data Socrata API

**Engineered features used downstream in ChromaDB retrieval:**
1. `composite_surge_score` — weighted mean of present / 2050s / 2080s storm surge FVI
2. `composite_tidal_score` — mean of available tidal FVI columns
3. `combined_flood_index` — 0.4 × surge + 0.3 × tidal + 0.3 × FSHRI
4. `flood_risk_tier` — ordinal label (1–5 → Very Low … Very High)
5. `crime_zscore` — standardized crime severity relative to borough mean
6. `overall_risk_score` — 0–100 composite of flood + crime features

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.patches import Patch

plt.rcParams['figure.dpi'] = 120
plt.rcParams['figure.figsize'] = (10, 5)

# Paths (run from project root)
FVI_CSV    = '../data/New_York_City_s_Flood_Vulnerability_Index.csv'
TRACTS_SHP = '../data/nyct2020_25a/nyct2020.shp'
BOROUGHS_SHP = '../data/nybb_25a/nybb.shp'

print('Libraries loaded.')

## 1. Load & Inspect FVI Data

In [ ]:
fvi = pd.read_csv(FVI_CSV)
fvi['GEOID'] = fvi['GEOID'].astype(str)

# Drop the embedded geometry column from the CSV (we use the shapefile instead)
if 'the_geom' in fvi.columns:
    fvi = fvi.drop(columns=['the_geom'])

print(f'Shape: {fvi.shape}')
fvi.head(3)

In [ ]:
fvi.describe().round(2)

## 2. Missing Value Analysis

Many tracts have `NaN` FVI scores — these are tracts outside current or projected flood zones.
We treat `NaN` as score = 0 (no flood risk) for feature engineering.

In [ ]:
fvi_cols = [
    'FSHRI',
    'FVI_storm_surge_present',
    'FVI_storm_surge_2050s',
    'FVI_storm_surge_2080s',
    'FVI_tidal_2020s',
    'FVI_tidal_2050s',
    'FVI_tidal_2080s',
]

missing = fvi[fvi_cols].isna().mean().sort_values(ascending=False)

fig, ax = plt.subplots(figsize=(9, 4))
missing.plot.barh(ax=ax, color='steelblue', edgecolor='white')
ax.set_xlabel('Fraction of tracts with missing data')
ax.set_title('Missing Values by FVI Column')
ax.axvline(0.5, color='red', linestyle='--', alpha=0.6, label='50% threshold')
ax.legend()
plt.tight_layout()
plt.show()

print(missing.to_string())

## 3. FVI Score Distributions

In [ ]:
surge_cols = ['FVI_storm_surge_present', 'FVI_storm_surge_2050s', 'FVI_storm_surge_2080s']
tidal_cols = ['FVI_tidal_2020s', 'FVI_tidal_2050s', 'FVI_tidal_2080s']

fig, axes = plt.subplots(1, 2, figsize=(13, 4))

for col in surge_cols:
    vals = fvi[col].dropna()
    axes[0].hist(vals, bins=5, alpha=0.6, label=col.replace('FVI_storm_surge_', ''), range=(0.5, 5.5))
axes[0].set_title('Storm Surge FVI Distribution (flood-zone tracts only)')
axes[0].set_xlabel('FVI Score (1=Very Low … 5=Very High)')
axes[0].set_ylabel('Number of census tracts')
axes[0].legend()

for col in tidal_cols:
    vals = fvi[col].dropna()
    axes[1].hist(vals, bins=5, alpha=0.6, label=col.replace('FVI_tidal_', ''), range=(0.5, 5.5))
axes[1].set_title('Tidal Flood FVI Distribution')
axes[1].set_xlabel('FVI Score')
axes[1].legend()

plt.tight_layout()
plt.show()

## 4. Merge with Census Tract Geometries

In [ ]:
tracts = gpd.read_file(TRACTS_SHP).to_crs('EPSG:4326')
tracts['GEOID'] = tracts['GEOID'].astype(str)

merged = tracts.merge(fvi, on='GEOID', how='left')
boroughs = gpd.read_file(BOROUGHS_SHP).to_crs('EPSG:4326')

print(f'Census tracts total : {len(tracts):,}')
print(f'Tracts with FVI data: {merged["FVI_storm_surge_2050s"].notna().sum():,}')
print(f'Tracts in flood zones: {merged["FVI_storm_surge_2050s"].notna().mean():.1%}')

## 5. Choropleth — Storm Surge Risk (2050s)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 9))

# Base layer: all tracts in light grey
merged.plot(ax=ax, color='#e8e8e8', edgecolor='white', linewidth=0.2)

# Flood-zone tracts colored by 2050s storm surge score
flood_tracts = merged[merged['FVI_storm_surge_2050s'].notna()]
flood_tracts.plot(
    column='FVI_storm_surge_2050s',
    ax=ax,
    cmap='RdYlBu_r',
    vmin=1, vmax=5,
    edgecolor='white',
    linewidth=0.2,
    legend=True,
    legend_kwds={'label': 'Storm Surge FVI (1=Low, 5=High)', 'shrink': 0.6},
)

# Borough outlines
boroughs.boundary.plot(ax=ax, color='#333333', linewidth=1.2)

ax.set_title('NYC Storm Surge Flood Vulnerability Index — 2050s Projection', fontsize=14)
ax.axis('off')
plt.tight_layout()
plt.show()

## 6. Correlation Analysis

In [ ]:
try:
    import seaborn as sns
    corr = fvi[fvi_cols].corr()
    fig, ax = plt.subplots(figsize=(8, 6))
    sns.heatmap(
        corr, annot=True, fmt='.2f', cmap='coolwarm',
        center=0, linewidths=0.5, ax=ax,
        xticklabels=[c.replace('FVI_', '') for c in fvi_cols],
        yticklabels=[c.replace('FVI_', '') for c in fvi_cols],
    )
    ax.set_title('FVI Feature Correlation Matrix')
    plt.tight_layout()
    plt.show()
except ImportError:
    # Fallback without seaborn
    corr = fvi[fvi_cols].corr().round(2)
    print(corr.to_string())

## 7. Feature Engineering

We engineer composite features that:
- Collapse multi-horizon FVI columns into single scores
- Combine flood + socioeconomic dimensions
- Provide normalized (0–1) scores for downstream retrieval ranking

In [ ]:
def safe_mean(df, cols):
    """Row-wise mean ignoring NaN, returns 0 where all values are NaN."""
    return df[cols].mean(axis=1).fillna(0)

# --- Feature 1: Composite storm surge score (fill NaN → 0 = no risk) -----
surge_cols = ['FVI_storm_surge_present', 'FVI_storm_surge_2050s', 'FVI_storm_surge_2080s']
merged['composite_surge'] = safe_mean(merged, surge_cols)

# --- Feature 2: Composite tidal score -----------------------------------
tidal_cols = ['FVI_tidal_2020s', 'FVI_tidal_2050s', 'FVI_tidal_2080s']
merged['composite_tidal'] = safe_mean(merged, tidal_cols)

# --- Feature 3: FSHRI (fill NaN → median) --------------------------------
fshri_median = merged['FSHRI'].median()
merged['FSHRI_filled'] = merged['FSHRI'].fillna(fshri_median)

# --- Feature 4: Combined flood index (weighted) --------------------------
# Weights reflect expert judgment: storm surge > tidal > socioeconomic
merged['combined_flood_index'] = (
    0.4 * merged['composite_surge'] +
    0.3 * merged['composite_tidal'] +
    0.3 * merged['FSHRI_filled']
)

# --- Feature 5: Normalized flood risk (0–1) --------------------------------
merged['flood_risk_norm'] = (merged['combined_flood_index'] - 1) / 4  # scale 1–5 → 0–1
merged['flood_risk_norm'] = merged['flood_risk_norm'].clip(0, 1)

# --- Feature 6: Risk tier (ordinal) ---------------------------------------
bins   = [-0.001, 0.2, 0.4, 0.6, 0.8, 1.001]
labels = ['Very Low', 'Low', 'Moderate', 'High', 'Very High']
merged['flood_risk_tier'] = pd.cut(
    merged['flood_risk_norm'], bins=bins, labels=labels
)

print('Engineered features added.')
merged[['GEOID', 'composite_surge', 'composite_tidal',
        'combined_flood_index', 'flood_risk_norm', 'flood_risk_tier']].head(8)

In [ ]:
tier_counts = merged['flood_risk_tier'].value_counts().reindex(labels)

colors = ['#2ecc71', '#a8d8a8', '#f39c12', '#e74c3c', '#8e1a1a']
fig, ax = plt.subplots(figsize=(9, 4))
tier_counts.plot.bar(ax=ax, color=colors, edgecolor='white')
ax.set_title('NYC Census Tract Distribution by Engineered Flood Risk Tier')
ax.set_xlabel('Risk Tier')
ax.set_ylabel('Number of census tracts')
ax.tick_params(axis='x', rotation=0)
plt.tight_layout()
plt.show()

## 8. Borough-Level Risk Summary

In [ ]:
# Map BoroCode from tract shapefile (NTAName contains borough info) or use BoroCT2020
boro_map = {'1': 'Manhattan', '2': 'Bronx', '3': 'Brooklyn', '4': 'Queens', '5': 'Staten Island'}
merged['borough'] = merged['GEOID'].str[2].map(boro_map)  # digit 3 is borough in NYC GEOID

boro_stats = merged.groupby('borough').agg(
    tracts=('GEOID', 'count'),
    mean_flood_index=('combined_flood_index', 'mean'),
    flood_zone_pct=('FVI_storm_surge_2050s', lambda x: x.notna().mean() * 100),
    mean_fshri=('FSHRI', 'mean'),
).round(2).sort_values('mean_flood_index', ascending=False)

print('Borough-level flood risk summary:')
display(boro_stats)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(13, 4))

boro_stats['mean_flood_index'].plot.bar(
    ax=axes[0], color='steelblue', edgecolor='white'
)
axes[0].set_title('Mean Combined Flood Index by Borough')
axes[0].set_ylabel('Combined Flood Index (1–5)')
axes[0].tick_params(axis='x', rotation=30)

boro_stats['flood_zone_pct'].plot.bar(
    ax=axes[1], color='coral', edgecolor='white'
)
axes[1].set_title('% of Tracts in a 2050s Storm Surge Flood Zone')
axes[1].set_ylabel('Percent (%)')
axes[1].tick_params(axis='x', rotation=30)

plt.tight_layout()
plt.show()

## 9. Crime Severity Feature Engineering

We use a pre-computed sample of Brooklyn crime severity scores (from `crime_agent.py`) to demonstrate the feature engineering without making live API calls here.

In [ ]:
# Representative Brooklyn severity scores (per 1,000 m²)
# Generated by crime_agent.sample_high_density_grid('BROOKLYN', required=35)
brooklyn_scores = np.array([
    1.05, 1.18, 0.87, 2.41, 1.67, 0.93, 3.12, 1.54, 2.08, 0.72,
    1.30, 2.55, 0.64, 1.89, 1.22, 0.98, 2.03, 1.45, 1.77, 0.81,
    1.60, 2.30, 0.69, 1.38, 1.91, 1.14, 2.78, 0.95, 1.52, 2.17,
    0.76, 1.43, 1.99, 1.26, 1.65,
])

boro_mean = brooklyn_scores.mean()
boro_std  = brooklyn_scores.std()

# Property-level scores for 3 example addresses
example_scores = {
    '621 Morgan Ave (Bushwick)': 1.22,
    '1 MetroTech Ctr (Downtown BK)': 2.45,
    '9 Ave V (Gravesend)': 0.68,
}

print(f'Borough mean  : {boro_mean:.2f}')
print(f'Borough std   : {boro_std:.2f}')
print()

# Z-score and risk tier
for addr, score in example_scores.items():
    zscore = (score - boro_mean) / boro_std
    tier = 'High' if zscore > 1 else ('Low' if zscore < -0.5 else 'Moderate')
    print(f'{addr}')
    print(f'  severity={score:.2f}  z-score={zscore:+.2f}  crime_risk_tier={tier}')

In [ ]:
fig, ax = plt.subplots(figsize=(9, 4))
ax.hist(brooklyn_scores, bins=12, color='slateblue', edgecolor='white', alpha=0.85)
ax.axvline(boro_mean, color='red', linestyle='--', linewidth=1.5, label=f'Mean = {boro_mean:.2f}')
ax.axvline(boro_mean + boro_std, color='orange', linestyle=':', linewidth=1.5,
           label=f'Mean ± 1σ = {boro_mean + boro_std:.2f}')
ax.axvline(boro_mean - boro_std, color='orange', linestyle=':', linewidth=1.5)

for addr, score in example_scores.items():
    ax.axvline(score, color='green', linestyle='-', linewidth=1, alpha=0.7)

ax.set_title('Brooklyn Crime Severity Score Distribution (sampled high-density grid)')
ax.set_xlabel('Severity Score (per 1,000 m²)')
ax.set_ylabel('Count')
ax.legend()
plt.tight_layout()
plt.show()

## 10. Final Feature Set for RAG Retrieval

These engineered features are embedded into ChromaDB document metadata.
At query time, the pipeline constructs a semantic query from the target
property's risk profile and retrieves the most contextually similar tracts
to provide grounding for the Mistral-7B LLM.

In [ ]:
# Final feature set stored in ChromaDB per census tract
rag_features = [
    'GEOID',
    'FSHRI',                  # Raw socioeconomic vulnerability score
    'FVI_storm_surge_2050s',  # Near-future storm surge risk
    'FVI_storm_surge_2080s',  # Long-term storm surge risk
    'composite_surge',        # Engineered: weighted surge mean
    'composite_tidal',        # Engineered: weighted tidal mean
    'combined_flood_index',   # Engineered: multi-factor composite
    'flood_risk_norm',        # Engineered: normalized 0–1
    'flood_risk_tier',        # Engineered: ordinal label
]

sample = merged[rag_features].dropna(subset=['FVI_storm_surge_2050s']).head(10)
print('Sample of engineered RAG features (flood-zone tracts):')
display(sample.reset_index(drop=True))

In [ ]:
# Show how the combined flood index correlates with individual FVI columns
fig, ax = plt.subplots(figsize=(8, 5))
flood_zone = merged[merged['FVI_storm_surge_2050s'].notna()]
ax.scatter(
    flood_zone['FVI_storm_surge_2050s'],
    flood_zone['combined_flood_index'],
    c=flood_zone['FSHRI'].fillna(3),
    cmap='RdYlGn_r',
    alpha=0.6, s=20, edgecolors='none'
)
sm = plt.cm.ScalarMappable(cmap='RdYlGn_r', norm=mcolors.Normalize(1, 5))
plt.colorbar(sm, ax=ax, label='FSHRI (socioeconomic vulnerability)')
ax.set_xlabel('FVI Storm Surge 2050s')
ax.set_ylabel('Combined Flood Index (engineered)')
ax.set_title('Combined Flood Index vs. 2050s Storm Surge — colored by FSHRI')
plt.tight_layout()
plt.show()

## Summary

| Feature | Source | Role in pipeline |
|---|---|---|
| `FVI_storm_surge_*` | FVI CSV | Raw flood vulnerability scores |
| `FVI_tidal_*` | FVI CSV | Tidal flood projections |
| `FSHRI` | FVI CSV | Community recovery capacity |
| `composite_surge` | **Engineered** | Collapsed multi-horizon surge |
| `composite_tidal` | **Engineered** | Collapsed multi-horizon tidal |
| `combined_flood_index` | **Engineered** | ChromaDB metadata + doc text |
| `flood_risk_tier` | **Engineered** | Semantic label in doc text |
| `crime_zscore` | **Engineered** | Relative crime exposure |

The engineered features are embedded as natural-language annotations in each
ChromaDB document, enabling the RAG pipeline to retrieve tracts that are
**semantically similar in risk profile** rather than just geographically nearby.